In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from grade import Exo1, Exo2
import hashlib

sns.set()

¿Sobre qué datos vamos a trabajar? Vamos a cargar en el siguiente bloque una serie y un data frame, de los archivos "serie.csv" y "dataframe.csv"

In [ ]:
serie = pd.read_csv("serie.csv").iloc[:,0]
df = pd.read_csv("dataframe.csv")

# Z-Score

Calcule el z-score para cada valor de la serie, y descarte los valores considerados atípicos cuando éste supera el valor 2.5. Para verificar si su respuesta es correcta, busque la suma de todos los valores que no descartó, haga un cast a un `int` (o lo que es lo mismo, "floor" al flotante que les quede) y pásenselo a la función Exo1.

In [ ]:
# Ordenamos la serie
serie_ord = serie.copy() # Hay que arreglar esto

# Calculamos el z-score
zscore = serie_ord+10 # Hay que arreglar esto

# Descartamos los valores que no nos interesan
zscore = zscore[zscore == 0] # Hay que arreglar esto

# Hacemos la suma de estos, y hacemos cast a un int
Exo1(int(0))

# IQR

Misma mentalidad, pero ahora calcule los cuartile susando `np.percentile` y `np.logical_and`

In [ ]:
Q1 = 0
Q3 = 1

IQR = Q3 - Q1

serie_iqr = serie[serie==0]

# Hacemos la suma de estos, y hacemos cast a un int
Exo2(int(0))

# DBSCAN

Contemple la distribución de puntos de df

In [ ]:
plt.plot(df.iloc[:,0], df.iloc[:,1], '.')

Como último ejercicio, haga la implementación correcta de la clase `DBSCAN`. Para ello, está la función auxiliar `euclidean`

In [ ]:
def euclidean(x, y):
    return np.linalg.norm(x - y)

El único método a implementar es la función `classify`, que toma como único argumento los datos, los guarda, y debe guardar los índices de los puntos clasificados como Núcleo, Frontera y valores atípicos.

In [ ]:
class DBSCAN:
    def __init__(self, epsilon, minPts, metric = euclidean):
        self.metric = metric
        self.kernelPoints = []
        self.frontierPoints = []
        self.outliers = []

    def classify(self, data):
        pass